In [1]:
import duckdb
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
import threading
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv()
thread_local = threading.local()

DB_PATHS = [
    "C:\\Users\\OSVALDO-SOFTENG\\Documents\\edward-portfolio\\GIT\\ai-data-analyzer\\olist.db",
    "/media/edward/SSD-Data/My Folder/ai-data-analyzer/olist.db"
]

# Thread-local storage for database connections
thread_local = threading.local()

def get_db_connection():
    """Get a thread-safe database connection."""
    if not hasattr(thread_local, "conn") or thread_local.conn is None:
        for db_path in DB_PATHS:
            try:
                conn = duckdb.connect(database=db_path, read_only=False)
                thread_local.conn = conn
                thread_local.db_path = db_path
                print(f" ! Database connected: {db_path}")
                conn.execute("LOAD spatial;")
                conn.execute("LOAD httpfs;")
                conn.execute("LOAD fts;")
                conn.execute("LOAD icu;")
                print(f" ! Spatial, HTTP, FTS, ICU loaded in database: {db_path}")
                break
            except Exception as e:
                print(f" ! Failed to connect to {db_path}: {e}")
                continue
        else:
            raise Exception("Failed to connect to any database path")
    return thread_local.conn

def get_table_name():
    conn = get_db_connection()
    result = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='main';").fetchall()
    table_names = [row[0] for row in result]
    return table_names

def get_column_types():
    conn = get_db_connection()
    result = conn.execute("SELECT table_name, column_name, data_type FROM information_schema.columns").fetchall()
    column_types = [(row[0], row[1], row[2]) for row in result]
    for table_name, column_name, data_type in column_types:
        print(f"- {table_name}.{column_name}: {data_type}")
    return column_types

def rel_db_relationship():
    return """
## Database Schema Relationships
- orders.customer_id = customers.customer_id
- orders.order_id = order_items.order_id
- orders.order_id = order_reviews.order_id
- orders.order_id = order_payments.order_id
- order_items.product_id = products.product_id
- order_items.seller_id = sellers.seller_id
- customers.customer_zip_code_prefix = geolocation.zip_code_prefix
- sellers.seller_zip_code_prefix = geolocation.zip_code_prefix

## Important Domain Notes
- customer_id is unique per order
- Use customer_unique_id to identify repeat customers
- Product categories are stored in Portuguese
- Use product_category_name_translation when English labels are required
"""

if __name__ == "__main__":
    conn = get_db_connection()
    table_names = print(f"Table names: {get_table_name()}")

    column_types = get_column_types()
    rel_db_relationship_info = rel_db_relationship()

    agent_name = "gpt-4o-mini-2024-07-18"
    agent_model = ChatOpenAI(
        model = agent_name,
        temperature = 0.2,
        max_tokens = 5000
    )

    

 ! Database connected: C:\Users\OSVALDO-SOFTENG\Documents\edward-portfolio\GIT\ai-data-analyzer\olist.db
 ! Spatial, HTTP, FTS, ICU loaded in database: C:\Users\OSVALDO-SOFTENG\Documents\edward-portfolio\GIT\ai-data-analyzer\olist.db
Table names: ['customers', 'geolocation', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers', 'product_category_translation']
- customers.customer_id: VARCHAR
- customers.customer_unique_id: VARCHAR
- customers.customer_zip_code_prefix: BIGINT
- customers.customer_city: VARCHAR
- customers.customer_state: VARCHAR
- geolocation.geolocation_zip_code_prefix: BIGINT
- geolocation.geolocation_lat: DOUBLE
- geolocation.geolocation_lng: DOUBLE
- geolocation.geolocation_city: VARCHAR
- geolocation.geolocation_state: VARCHAR
- order_items.order_id: VARCHAR
- order_items.order_item_id: BIGINT
- order_items.product_id: VARCHAR
- order_items.seller_id: VARCHAR
- order_items.shipping_limit_date: VARCHAR
- order_items.price: DOUBLE
- order

In [2]:
conn.execute("SELECT table_name, column_name, data_type FROM information_schema.columns").fetchall()

[('customers', 'customer_id', 'VARCHAR'),
 ('customers', 'customer_unique_id', 'VARCHAR'),
 ('customers', 'customer_zip_code_prefix', 'BIGINT'),
 ('customers', 'customer_city', 'VARCHAR'),
 ('customers', 'customer_state', 'VARCHAR'),
 ('geolocation', 'geolocation_zip_code_prefix', 'BIGINT'),
 ('geolocation', 'geolocation_lat', 'DOUBLE'),
 ('geolocation', 'geolocation_lng', 'DOUBLE'),
 ('geolocation', 'geolocation_city', 'VARCHAR'),
 ('geolocation', 'geolocation_state', 'VARCHAR'),
 ('order_items', 'order_id', 'VARCHAR'),
 ('order_items', 'order_item_id', 'BIGINT'),
 ('order_items', 'product_id', 'VARCHAR'),
 ('order_items', 'seller_id', 'VARCHAR'),
 ('order_items', 'shipping_limit_date', 'VARCHAR'),
 ('order_items', 'price', 'DOUBLE'),
 ('order_items', 'freight_value', 'DOUBLE'),
 ('order_payments', 'order_id', 'VARCHAR'),
 ('order_payments', 'payment_sequential', 'BIGINT'),
 ('order_payments', 'payment_type', 'VARCHAR'),
 ('order_payments', 'payment_installments', 'BIGINT'),
 ('order_

In [3]:
conn.execute("""
             SELECT
LEFT(c.customer_unique_id, 10) AS customer_id,
LEFT(s.seller_id, 10) AS seller_id,
COUNT(o.order_id) AS total_orders,
AVG(ST_Distance(
ST_MakePoint(g_c.geolocation_lng, g_c.geolocation_lat),
ST_MakePoint(g_s.geolocation_lng, g_s.geolocation_lat)
)) AS avg_distance
FROM
customers c
JOIN
orders o ON c.customer_id = o.customer_id
JOIN
order_items oi ON o.order_id = oi.order_id
JOIN
sellers s ON oi.seller_id = s.seller_id
JOIN
geolocation g_c ON c.customer_zip_code_prefix = g_c.geolocation_zip_code_prefix
JOIN
geolocation g_s ON s.seller_zip_code_prefix = g_s.geolocation_zip_code_prefix
GROUP BY
c.customer_unique_id, s.seller_id
LIMIT 100;
""").fetchall()

[('efdde19079', '9646c35132', 4374, 0.843001747623442),
 ('d12a206aee', 'f464906244', 689, 0.16956385857894538),
 ('ff60aefe21', 'dbc2212516', 2769, 1.3580964075234498),
 ('964eb1b987', '55f7a3319d', 1568, 0.15330010203860756),
 ('d4acb6f403', 'fa1c13f261', 2920, 0.9628034052292312),
 ('84e049bcf3', '43402bdf22', 1591, 0.2939433630342429),
 ('f62a0996df', 'ea8482cd71', 945, 0.1022394485966464),
 ('935926295b', '4de6e4ba57', 2200, 3.1567565807464257),
 ('60a4bc62e8', 'd20b021d3e', 12870, 2.8284124793013437),
 ('b7de6c9f69', '6d66611d7c', 1248, 0.10983556915234764),
 ('69de2f5d39', '1b7e5006cb', 640, 0.09405009044661847),
 ('0220755d57', '52454f90d4', 4914, 0.7768083657044711),
 ('72c4723cc6', 'e38db88540', 2820, 7.872020757143484),
 ('6e96c18528', '4869f7a5df', 3570, 2.7013450713724216),
 ('f4f0ab82bf', '1025f0e2d4', 2268, 0.12306751799373482),
 ('d55e256d29', '2528513dd9', 6290, 0.13376322807596014),
 ('0894227c11', '620c87c171', 18330, 3.5864908742740784),
 ('e3084d5aa2', '93b9a99027'